<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/Linearization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [261]:
import jax
import jax.numpy as jnp
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp
from scipy.optimize import root,minimize
import scipy.signal as sig
jax.config.update('jax_enable_x64',True)

In [489]:
g=9.81
Aout=0.02
Ac=2
qin0 = 0.2
tend=500

In [508]:
def dhdt(t,v,qin):
    h=v[0]
    return jnp.array([(qin-Aout*jnp.sqrt(2*g*h))/Ac])

In [509]:
h0=root(lambda h: dhdt(0,h,qin0), [1.]).x

In [510]:
h0

array([5.09683996])

In [501]:
res=solve_ivp(dhdt,(0,tend),h0,args=(qin0*1.1,),method='LSODA',dense_output=True)

In [502]:
tplot=jnp.linspace(0,tend,200)
fig=make_subplots()
fig.add_scatter(x=tplot, y=res.sol(tplot)[0],name='nonlinear')
fig.update_layout(width=600,height=400,template='plotly_dark')

In [503]:
jac=jax.jacobian(dhdt,(1,2))

In [504]:
A,B=jac(0,h0,qin0)

In [505]:
sys=sig.lti(A,B,[1],[0])

In [506]:
_,yout,_=sig.lsim(sys,jnp.full_like(tplot,qin0*0.1),tplot)

In [507]:
tplot=jnp.linspace(0,tend,200)
fig=make_subplots()
fig.add_scatter(x=tplot, y=res.sol(tplot)[0],name='nonlinear')
fig.add_scatter(x=tplot, y=yout+h0,name='linearized')
fig.update_layout(width=600,height=400,template='plotly_dark')